# MoS₂ — abTEM Multislice Simulation

Full multislice simulation of MoS₂ [001] zone axis using [abTEM](https://abtem.github.io/abtem/):
- **4D-STEM** with PixelatedDetector → interactive exploration with Show4DSTEM
- **HAADF-STEM** with AnnularDetector → specimen drift simulation → alignment with Align2D

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass  # autoreload unavailable (Colab Python 3.12+)

In [2]:
import abtem
import numpy as np
from ase.io import read
from quantem.widget import Align2D, Show2D, Show4DSTEM

## Crystal structure

Read MoS₂ from CIF, orthogonalize the hexagonal cell (γ=120° → 90°), and create a supercell.

In [3]:
atoms = read("/Users/macbook/repos/quantem.widget/data/MoS2.cif")
atoms_ortho = abtem.orthogonalize_cell(atoms)
print(f"Formula: {atoms.get_chemical_formula()}")
print(f"Original cell: a={atoms.cell.lengths()[0]:.4f} Å, γ={atoms.cell.angles()[2]:.0f}°")
print(f"Orthogonal cell: {atoms_ortho.cell.lengths()[0]:.4f} × {atoms_ortho.cell.lengths()[1]:.4f} Å")

slab = atoms_ortho.repeat((6, 4, 1))
Lx, Ly = slab.cell.lengths()[0], slab.cell.lengths()[1]
print(f"Supercell: {len(slab)} atoms, {Lx:.1f} × {Ly:.1f} Å")

Formula: Mo2S4
Original cell: a=3.1854 Å, γ=120°
Orthogonal cell: 3.1854 × 5.5173 Å
Supercell: 288 atoms, 19.1 × 22.1 Å


## Multislice setup

300 kV electron beam with 21.4 mrad convergence semi-angle.
Potential sampled at 0.05 Å for accurate multislice propagation.

In [4]:
potential = abtem.Potential(slab, sampling=0.05, slice_thickness=1.0)
probe = abtem.Probe(energy=300e3, semiangle_cutoff=21.4, sampling=potential.sampling)

print(f"Potential grid: {potential.gpts}")
print(f"Sampling: {potential.sampling[0]:.4f} × {potential.sampling[1]:.4f} Å")
print(f"Probe: 300 kV, convergence α = 21.4 mrad")

Potential grid: (383, 442)
Sampling: 0.0499 × 0.0499 Å
Probe: 300 kV, convergence α = 21.4 mrad


## 4D-STEM simulation

Capture the full convergent beam electron diffraction (CBED) pattern at each scan position
using a pixelated detector. Coarse scan step (0.5 Å) keeps data size manageable.

In [5]:
max_angle = 120  # mrad
scan_sampling_4d = 0.5  # Å

scan_4d = abtem.GridScan(start=(0, 0), end=(Lx, Ly), sampling=scan_sampling_4d)
detector_4d = abtem.PixelatedDetector(max_angle=max_angle)

print(f"Running 4D-STEM scan (sampling={scan_sampling_4d} Å, max_angle={max_angle} mrad)...")
measurement_4d = probe.scan(potential, scan=scan_4d, detectors=detector_4d)
data_4d = np.asarray(measurement_4d.array, dtype=np.float32)

print(f"Shape: {data_4d.shape}")
print(f"  Scan: {data_4d.shape[0]} × {data_4d.shape[1]} positions")
print(f"  Detector: {data_4d.shape[2]} × {data_4d.shape[3]} pixels")
print(f"  Memory: {data_4d.nbytes / 1e6:.0f} MB")

# k-space calibration: reciprocal space pixel = λ / cell_length
wavelength = 0.01969  # Å at 300 kV
k_px = wavelength * 1000 / Lx  # mrad/px
print(f"  Real-space: {scan_sampling_4d} Å/px, k-space: {k_px:.3f} mrad/px")

Running 4D-STEM scan (sampling=0.5 Å, max_angle=120 mrad)...
Shape: (39, 45, 235, 272)
  Scan: 39 × 45 positions
  Detector: 235 × 272 pixels
  Memory: 449 MB
  Real-space: 0.5 Å/px, k-space: 1.030 mrad/px


In [6]:
w4d = Show4DSTEM(data_4d, pixel_size=scan_sampling_4d, k_pixel_size=k_px)
w4d.auto_detect_center()
w4d.roi_circle()
print(f"BF disk: center=({w4d.center_row:.1f}, {w4d.center_col:.1f}), radius={w4d.bf_radius:.1f} px")
w4d

BF disk: center=(117.0, 136.0), radius=22.3 px


Show4DSTEM(shape=(39, 45, 235, 272), sampling=(0.5 Å, 1.0302107680532273 mrad), pos=(19, 22))

## HAADF-STEM simulation

High-angle annular dark field: integrate scattered intensity from 68–100 mrad (Z-contrast).
Finer scan step (0.2 Å) for atomic-resolution imaging of Mo and S columns.

In [7]:
scan_sampling_haadf = 0.2  # Å

scan_haadf = abtem.GridScan(start=(0, 0), end=(Lx, Ly), sampling=scan_sampling_haadf)
detector_haadf = abtem.AnnularDetector(inner=68, outer=100)

print(f"Running HAADF scan (sampling={scan_sampling_haadf} Å, detector: 68–100 mrad)...")
measurement_haadf = probe.scan(potential, scan=scan_haadf, detectors=detector_haadf)
haadf = np.asarray(measurement_haadf.array, dtype=np.float32)

print(f"HAADF shape: {haadf.shape}")
print(f"Range: [{haadf.min():.6f}, {haadf.max():.6f}]")

Running HAADF scan (sampling=0.2 Å, detector: 68–100 mrad)...
HAADF shape: (96, 111)
Range: [0.000048, 0.019281]


In [8]:
Show2D(haadf, title="MoS₂ HAADF (abTEM multislice)", pixel_size_angstrom=scan_sampling_haadf)

## Image registration with Align2D

Simulate specimen drift between consecutive STEM acquisitions:
sub-pixel shift + independent Poisson shot noise + readout noise.

In [9]:
from scipy.ndimage import shift as ndi_shift

rng = np.random.default_rng(42)


def make_noisy_pair(img, drift_row, drift_col, dose=5e3, readout=0.02):
    """Create a pair of noisy images with sub-pixel drift."""
    img_norm = (img - img.min()) / (img.max() - img.min())
    counts = img_norm * dose

    a = rng.poisson(np.maximum(counts, 0)).astype(np.float32)
    a += rng.normal(0, readout * dose, a.shape).astype(np.float32)

    shifted = ndi_shift(counts, (drift_row, drift_col), order=3, mode="reflect")
    b = rng.poisson(np.maximum(shifted, 0)).astype(np.float32)
    b += rng.normal(0, readout * dose, b.shape).astype(np.float32)

    return a, b


drift = (3.4, -2.1)  # px
img_a, img_b = make_noisy_pair(haadf, *drift)
print(f"True drift: ({drift[0]:+.1f}, {drift[1]:+.1f}) px = ({drift[0] * scan_sampling_haadf:+.2f}, {drift[1] * scan_sampling_haadf:+.2f}) Å")

True drift: (+3.4, -2.1) px = (+0.68, -0.42) Å


In [10]:
Show2D(
    [img_a, img_b],
    title="HAADF pair with drift",
    labels=["Frame A", "Frame B"],
    pixel_size_angstrom=scan_sampling_haadf,
)

In [11]:
w = Align2D(
    img_a,
    img_b,
    title="MoS₂ HAADF — abTEM",
    label_a="Frame A",
    label_b="Frame B (drifted)",
    pixel_size=scan_sampling_haadf / 10,  # nm
)
w

In [12]:
dx, dy = w.offset
print(f"True drift:     ({drift[0]:+.2f}, {drift[1]:+.2f}) px")
print(f"Detected shift: (dx={dx:+.2f}, dy={dy:+.2f}) px")
print(f"NCC: {w.xcorr_zero:.3f} (before) → {w.ncc_aligned:.3f} (after)")

True drift:     (+3.40, -2.10) px
Detected shift: (dx=+2.07, dy=-3.42) px
NCC: -0.034 (before) → 0.669 (after)
